In [1]:
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [3]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [ ]:
# load dataset
dataset = read_csv('../input/lstm-datasets-multivariate-univariate/LSTM-Multivariate_pollution.csv', header=0, index_col=0)
values = dataset.values

In [3]:
import requests

access_key = '0c0e878c-9418-4e9a-b43d-47155a8101c4'

headers = {
    'X-Meteum-API-Key': access_key
}

response = requests.get('https://api.meteum.ai/v1/forecast?lat=52.37125&lon=4.89388', headers=headers)

print(response.json())

{'now': 1690259017, 'now_dt': '2023-07-25T04:23:37.880415Z', 'info': {'n': True, 'geoid': 10466, 'url': 'https://meteum.ai/10466?lat=52.37125&lon=4.89388', 'lat': 52.37125, 'lon': 4.89388, 'tzinfo': {'name': 'Europe/Amsterdam', 'abbr': 'CEST', 'dst': True, 'offset': 7200}, 'def_pressure_mm': 759, 'def_pressure_pa': 1011, 'slug': '10466', 'zoom': 10, 'nr': True, 'ns': True, 'nsr': True, 'p': False, 'f': True, '_h': False}, 'geo_object': {'district': None, 'locality': {'id': 10466, 'name': 'Amsterdam'}, 'province': {'id': 118, 'name': 'North Holland'}, 'country': {'id': 118, 'name': 'Netherlands'}}, 'yesterday': {'temp': 17}, 'fact': {'obs_time': 1690257600, 'uptime': 1690259017, 'temp': 13, 'feels_like': 11, 'temp_water': 7, 'icon': 'ovc', 'condition': 'overcast', 'cloudness': 1, 'prec_type': 0, 'prec_prob': 90, 'prec_strength': 0, 'is_thunder': False, 'wind_speed': 3.6, 'wind_dir': 'n', 'pressure_mm': 755, 'pressure_pa': 1006, 'humidity': 82, 'daytime': 'd', 'polar': False, 'season': '

In [5]:
response.json()

{'now': 1690259017,
 'now_dt': '2023-07-25T04:23:37.880415Z',
 'info': {'n': True,
  'geoid': 10466,
  'url': 'https://meteum.ai/10466?lat=52.37125&lon=4.89388',
  'lat': 52.37125,
  'lon': 4.89388,
  'tzinfo': {'name': 'Europe/Amsterdam',
   'abbr': 'CEST',
   'dst': True,
   'offset': 7200},
  'def_pressure_mm': 759,
  'def_pressure_pa': 1011,
  'slug': '10466',
  'zoom': 10,
  'nr': True,
  'ns': True,
  'nsr': True,
  'p': False,
  'f': True,
  '_h': False},
 'geo_object': {'district': None,
  'locality': {'id': 10466, 'name': 'Amsterdam'},
  'province': {'id': 118, 'name': 'North Holland'},
  'country': {'id': 118, 'name': 'Netherlands'}},
 'yesterday': {'temp': 17},
 'fact': {'obs_time': 1690257600,
  'uptime': 1690259017,
  'temp': 13,
  'feels_like': 11,
  'temp_water': 7,
  'icon': 'ovc',
  'condition': 'overcast',
  'cloudness': 1,
  'prec_type': 0,
  'prec_prob': 90,
  'prec_strength': 0,
  'is_thunder': False,
  'wind_speed': 3.6,
  'wind_dir': 'n',
  'pressure_mm': 755,
  